## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Clinical De-identification - Portuguese](https://aws.amazon.com/marketplace/pp/prodview-toznqfuzuf35q).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification Portuguese

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: [pt.deid.clinical](https://nlp.johnsnowlabs.com/2022/06/21/clinical_deidentification_pt_3_0.html)
- **Model Description**: This pipeline is trained with w2v_cc_300d portuguese embeddings and can be used to deidentify PHI information from medical texts in portuguese. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask, fake or obfuscate the following entities: AGE, DATE, PROFESSION, EMAIL, ID, COUNTRY, STREET, DOCTOR, HOSPITAL, PATIENT, URL, IP, ORGANIZATION, PHONE, ZIP, ACCOUNT, SSN, PLATE, SEX and IPADDR

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_name = "pt-deid-clinical"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

### Important Parameter

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    Example: "**Dados do paciente. Nome: Mauro. Apelido: Gonçalves. NISS: 26 63514095. Endereço: Calle Miguel Benitez 90.**"

    - **masked**: Default policy that masks entities with their type.

      -> 'Dados do `<SEX>`. Nome: `<PATIENT>`. Apelido: `<PATIENT>`. NISS: `<ID>`. Endereço: `<STREET>`.'

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      -> 'Dados do `Homen`. Nome: `Josefina Batista`. Apelido: `Tiago Santos`. NISS: `16 63514095`. Endereço: `Avenida da Liberdade, 17`.'

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

      -> 'Dados do `****`. Nome: `****`. Apelido: `****`. NISS: `****`. Endereço: `****`.'

    - **masked_with_chars**: Masks entities with asterisks (\*).

      -> 'Dados do [`******`]. Nome: [`***`]. Apelido: [`*******`]. NISS: [`**********`]. Endereço: [`*********************`].'
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked"
}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [8]:
docs = [
'''Dados do paciente.
Nome: Mauro.
Apelido: Gonçalves.
NISS: 26 63514095.
Endereço: Calle Miguel Benitez 90.
CÓDIGO POSTAL: 28016.
Dados de cuidados.
Data de nascimento: 03/03/1946.
País: Portugal.
Idade: 70 anos Sexo: M.
Data de admissão: 12/12/2016.
Doutor: Ignacio Navarro Cuéllar NºCol: 28 28 70973.
Relatório clínico do paciente: Paciente de 70 anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia.
Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.
O exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV.
A urinálise mostrou 4 glóbulos vermelhos/campo e 0-5 leucócitos/campo; o resto do sedimento estava normal.
Hemograma normal; a bioquímica mostrou glicemia de 169 mg/dl e triglicéridos de 456 mg/dl; função hepática e renal normal. PSA de 1,16 ng/ml.
A citologia da urina era repetidamente desconfiada por malignidade.
A radiografia simples abdominal mostra alterações degenerativas na coluna lombar e calcificações vasculares tanto no hipocôndrio como na pélvis.
A ecografia urológica revelou cistos corticais simples no rim direito, uma bexiga inalterada com boa capacidade e uma próstata com 30g de peso.
O IVUS mostrou normofuncionalismo renal bilateral, calcificações na silhueta renal direita e ureteres artrosados com imagens de adição no terço superior de ambos os ureteres, relacionadas com pseudodiverticulose ureteral. O cistograma mostra uma bexiga com boa capacidade, mas com paredes trabeculadas em relação à bexiga de stress. A tomografia computorizada abdominal é normal.
A cistoscopia revelou a existência de pequenos tumores na bexiga, e a ressecção transuretral foi realizada com o resultado anatomopatológico do carcinoma urotelial superficial da bexiga.
Referido por: Miguel Santos - Avenida dos Aliados, 22 Portugal E-mail: nnavcu@hotmail.com.''',

'''RELAÇÃO HOSPITALAR
NOME: Pedro Gonçalves
NHC: MVANSK92F09W408A
ENDEREÇO: Rua Burcardo 7
CÓDIGO POSTA... 
RELAÇÃO HOSPITALAR
NOME: Pedro Gonçalves
NHC: MVANSK92F09W408A
ENDEREÇO: Rua Burcardo 7
CÓDIGO POSTAL: 80139
DATA DE NASCIMENTO: 03/03/1946
IDADE: 70 anos
SEXO: Homens
E-MAIL: pgon21@tim.pt
DATA DE ADMISSÃO: 12/12/2016
DOUTORA: Eva Andrade
RELATO CLÍNICO: 70 anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia.
Ele foi-nos encaminhado por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.
O exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV.
A urinálise mostrou 4 hemácias/campo e 0-5 leucócitos/campo; o resto do sedimento era normal.
O hemograma é normal; a bioquímica mostrou uma glicemia de 169 mg/dl e triglicerídeos 456 mg/dl; função hepática e renal são normais. PSA de 1,16 ng/ml.

DIRIGIDA A: Dr. Eva Andrade - Centro Hospitalar do Medio Ave - Avenida Dos Aliados, 56
E-MAIL: evandrade@poste.pt'''

]


sample_text = """Nome do paciente: Antonio Gonçalves
NHC: 2569870.
Endereço: Rua Das Flores 23.
Código Postal: 21754-... 
Nome do paciente: Antonio Gonçalves
NHC: 2569870.
Endereço: Rua Das Flores 23.
Código Postal: 21754-987.
Dados de cuidados.
Data de nascimento: 10/10/1963.
Idade: 23 anos 
Data de admissão: 17/06/2016.
Doutora: Maria Santos"""

### JSON

#### Example 1: masked (default-policy)

In [9]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
print(response_json["predictions"][0])

Nome do <SEX>: <PATIENT>
NHC: <ID>.
Endereço: <STREET>.
Código Postal: <ZIP>-... 
Nome do <SEX>: <PATIENT>
NHC: <ID>.
Endereço: <STREET>.
Código Postal: <ZIP>.
Dados de cuidados.
Data de nascimento: <DATE>.
Idade: <AGE> anos 
Data de admissão: <DATE>.
Doutora: <DOCTOR>


#### Example 2: obfuscated

In [11]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
print(response_json["predictions"][0])

Nome do Homen: Silvia Couto
NHC: 7856923.
Endereço: Avenida da Liberdade, 17.
Código Postal: 70281-... 
Nome do Homen: Silvia Couto
NHC: 7856923.
Endereço: Avenida da Liberdade, 17.
Código Postal: 70281-692.
Dados de cuidados.
Data de nascimento: 10/10/1963.
Idade: 27 anos 
Data de admissão: 17/06/2016.
Doutora: Isabel Magalhães


### JSON Lines

In [13]:
def create_jsonl(records, masking_policy=None):
    json_records = []

    if isinstance(records, str):
        records = [records]

    for text in records:
        record = {"text": text}

        if masking_policy is not None:
            record["masking_policy"] = masking_policy
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record, ensure_ascii=False) for record in json_records)
    return json_lines


#### Example 1: masked (default-policy)

In [14]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="masked")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "Nome do <SEX>: <PATIENT>\nNHC: <ID>.\nEndereço: <STREET>.\nCódigo Postal: <ZIP>-... \nNome do <SEX>: <PATIENT>\nNHC: <ID>.\nEndereço: <STREET>.\nCódigo Postal: <ZIP>.\nDados de cuidados.\nData de nascimento: <DATE>.\nIdade: <AGE> anos \nData de admissão: <DATE>.\nDoutora: <DOCTOR>"}


#### Example 2: obfuscated

In [16]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="obfuscated")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "Nome do Homen: Silvia Couto\nNHC: 7856923.\nEndereço: Avenida da Liberdade, 17.\nCódigo Postal: 70281-... \nNome do Homen: Silvia Couto\nNHC: 7856923.\nEndereço: Avenida da Liberdade, 17.\nCódigo Postal: 70281-692.\nDados de cuidados.\nData de nascimento: 10/10/1963.\nIdade: 27 anos \nData de admissão: 17/06/2016.\nDoutora: Isabel Magalhães"}


### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [19]:
validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [20]:
# Create JSON and JSON Lines data
input_json_data = {
    "input1.json": json.dumps({"text": docs, "masking_policy": "masked"}, ensure_ascii=False),
    "input2.json": json.dumps({"text": docs, "masking_policy": "obfuscated"}, ensure_ascii=False),
    "input3.json": json.dumps({"text": docs, "masking_policy": "masked_fixed_length_chars"}, ensure_ascii=False),
    "input4.json": json.dumps({"text": docs, "masking_policy": "masked_with_chars"}, ensure_ascii=False),
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
    "input3.jsonl": create_jsonl(docs, masking_policy="masked_fixed_length_chars"),
    "input4.jsonl": create_jsonl(docs, masking_policy="masked_with_chars")
}

# Upload JSON and JSON Lines data to S3
for file_name, data in input_json_data.items():
    upload_to_s3(data, file_name)

for file_name, data in input_jsonl_data.items():
    upload_to_s3(data, file_name)


### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [22]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [23]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
    "masked_fixed_length_chars": "input3.json",
    "masked_with_chars": "input4.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print("-"*50, policy_name ,"-"*50)
    retrieve_json_output_from_s3(validation_file_name)
    print("\n")

-------------------------------------------------- masked --------------------------------------------------


{'predictions': ['Dados do <SEX>.\nNome: <PATIENT>.\nApelido: <PATIENT>.\nNISS: <PHONE> 63514095.\nEndereço: <STREET>.\nCÓDIGO POSTAL: <ZIP>.\nDados de cuidados.\nData de nascimento: <DATE>.\nPaís: <COUNTRY>.\nIdade: <AGE> anos Sexo: <SEX>.\nData de admissão: <DATE>.\nDoutor: <DOCTOR> Cuéllar NºCol: <ID> <ID> <ID>.\nRelatório clínico do <SEX>: <SEX> de <AGE> anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia.\nFoi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de p



-------------------------------------------------- obfuscated --------------------------------------------------


{'predictions': ['Dados do Homen.\nNome: Josefina Batista.\nApelido: Tiago Santos.\nNISS: 75 63514095.\nEndereço: Avenida da Liberdade, 17.\nCÓDIGO POSTAL: 79305.\nDados de cuidados.\nData de nascimento: 03/03/1946.\nPaís: Brasil.\nIdade: 77 anos Sexo: H..\nData de admissão: 12/12/2016.\nDoutor: Vasco Soares Cuéllar NºCol: 79 79 23624.\nRelatório clínico do Homen: Homen de 77 anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia.\nFoi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatíve



-------------------------------------------------- masked_fixed_length_chars --------------------------------------------------


{'predictions': ['Dados do ****.\nNome: ****.\nApelido: ****.\nNISS: **** 63514095.\nEndereço: ****.\nCÓDIGO POSTAL: ****.\nDados de cuidados.\nData de nascimento: ****.\nPaís: ****.\nIdade: **** anos Sexo: ****.\nData de admissão: ****.\nDoutor: **** Cuéllar NºCol: **** **** ****.\nRelatório clínico do ****: **** de **** anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia.\nFoi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV.\nA urinálise mo



-------------------------------------------------- masked_with_chars --------------------------------------------------


{'predictions': ['Dados do [******].\nNome: [***].\nApelido: [*******].\nNISS: ** 63514095.\nEndereço: [*********************].\nCÓDIGO POSTAL: [***].\nDados de cuidados.\nData de nascimento: [********].\nPaís: [******].\nIdade: ** anos Sexo: *.\nData de admissão: [********].\nDoutor: [*************] Cuéllar NºCol: ** ** [***].\nRelatório clínico do [******]: [******] de ** anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia.\nFoi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível 

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [25]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [26]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
    "masked_fixed_length_chars": "input3.jsonl",
    "masked_with_chars": "input4.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print("-"*50, policy_name ,"-"*50)
    retrieve_jsonlines_output_from_s3(validation_file_name)
    print("\n")

-------------------------------------------------- masked --------------------------------------------------
{"predictions": "Dados do <SEX>.\nNome: <PATIENT>.\nApelido: <PATIENT>.\nNISS: <PHONE> 63514095.\nEndereço: <STREET>.\nCÓDIGO POSTAL: <ZIP>.\nDados de cuidados.\nData de nascimento: <DATE>.\nPaís: <COUNTRY>.\nIdade: <AGE> anos Sexo: <SEX>.\nData de admissão: <DATE>.\nDoutor: <DOCTOR> Cuéllar NºCol: <ID> <ID> <ID>.\nRelatório clínico do <SEX>: <SEX> de <AGE> anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia.\nFoi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.\nO exame físico 

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

